In [ ]:
# import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from ecig_parsing import *


In [ ]:
def get_html(url, clicked=False, closed=False, elements=True):
    # print('GET', url)
    try:
        # Set up headless mode for Firefox
        options = Options()
        options.add_argument('--headless')  # Enable headless mode properly

        # Start the WebDriver in headless mode
        driver = webdriver.Firefox(options=options)

        driver.get(url)
        last_n = 0
        hrefs = set()
        button_id = 'ac-ag-yes-button'

        html = driver.page_source
        time.sleep(1)
        same_count = 0
#         if not clicked:
#             wait = WebDriverWait(driver, 10)  # 10 seconds timeout
#             button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))  # Use ID, XPATH, or other locator

#             # Click the button
#             button.click()
#             clicked = True
#         if not closed:
#             wait = WebDriverWait(driver, 10)  # 10 seconds timeout
#             button = wait.until(EC.element_to_be_clickable((By.ID, 'omnisend-form-65006a9819ed2c118d9a9352-close-action')))  # Use ID, XPATH, or other locator

#             # Click the button
#             button.click()
#             closed = True
    

        # Set a delay for dynamic content loading
        SCROLL_PAUSE_TIME = 0.5  # Shorter pause for smoother scroll

        # Scroll incrementally by a small step (e.g., 500 pixels)
        scroll_increment = 500

        # Get the initial scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        first_height = last_height
        # print(last_height)

        while True:
            # Scroll down by the increment
            driver.execute_script(f"window.scrollBy(0, {scroll_increment});")

            # Wait for the page to load new content
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate the new scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # Check if new elements have appeared on the screen
            # You can check specific elements like this:
            # elements = driver.find_elements(By.CLASS_NAME, "your-element-class")

            # If the scroll height has not changed, break the loop
            #print(same_count, new_height, last_height)
            if new_height == last_height :
                same_count += 1
                if same_count >= 3:
                    break
            else:
                same_count = 0

            # Update the last height
            last_height = new_height
            
            html = driver.page_source
            #print(html)
        

    except Exception as ex:
        print(ex)
        html = ''
    finally:
        if driver:
            driver.close()




    return html


In [ ]:
links = ['https://perfectvape.com/10-off-disposables/', 'https://perfectvape.com/20-off-overstock/', 
         'https://perfectvape.com/deals/', 'https://perfectvape.com/disposables-1/', 
         'https://perfectvape.com/disposables-1/', 'https://perfectvape.com/0-nicotine-disposable-vapes/', 
         'https://perfectvape.com/3-nicotine-disposable-vapes/', 
         'https://perfectvape.com/4-nicotine-disposable-vapes/', 
         'https://perfectvape.com/5-nicotine-disposable-vapes/', 'https://perfectvape.com/6/', 'https://perfectvape.com/6-8/',
         'https://perfectvape.com/pmta-exempt/', 'https://perfectvape.com/rechargeable-disposable-vapes', 
         'https://perfectvape.com/smart-disposable-vapes', 'https://perfectvape.com/new-hot-items/', 
         'https://perfectvape.com/nicotine-pouches/', 'https://perfectvape.com/tobacco-free-tfn/', 
         'https://perfectvape.com/tobacco-free-tfn/', 'https://perfectvape.com/tobacco-free-tfn-e-liquids/', 'https://perfectvape.com/tobacco-free-tfn-disposables/', 'https://perfectvape.com/premium-e-liquids/', 'https://perfectvape.com/premium-e-liquids/',
         'https://perfectvape.com/regular-nicotine/', 'https://perfectvape.com/regular-nicotine/', 
         'https://perfectvape.com/air-factory/', 'https://perfectvape.com/candy-king/',
         'https://perfectvape.com/jam-monster/', 'https://perfectvape.com/mucho/', 'https://perfectvape.com/hi-drip/', 'https://perfectvape.com/teardrip/', 'https://perfectvape.com/tobacco-free-nicotine-tfn-e-liquids/', 'https://perfectvape.com/salt-nic/', 'https://perfectvape.com/salt-nic/', 'https://perfectvape.com/candy-king-on-salt/', 'https://perfectvape.com/duo/', 'https://perfectvape.com/hi-drip-1/', 'https://perfectvape.com/jam-monster-salt-nic/', 'https://perfectvape.com/tropic-king-on-salt/', 
         'https://perfectvape.com/vapor-products/', 'https://perfectvape.com/vapor-products/', 
         'https://perfectvape.com/e-liquid-juices/', 'https://perfectvape.com/e-liquid-juices/', 
         'https://perfectvape.com/bad-drip/', 'https://perfectvape.com/burst/', 'https://perfectvape.com/fryd/',
         'https://perfectvape.com/candy-king-3/', 'https://perfectvape.com/cloud-nurdz-2/', 'https://perfectvape.com/fruitia/', 'https://perfectvape.com/glas/',
         'https://perfectvape.com/humble-juice-1/', 'https://perfectvape.com/coastal-clouds/', 'https://perfectvape.com/hyde-2/', 'https://perfectvape.com/juice-head/', 
         'https://perfectvape.com/mints/', 'https://perfectvape.com/monster-vape-labs/', 'https://perfectvape.com/naked/', 'https://perfectvape.com/pacha-mama/', 
         'https://perfectvape.com/perfect-vape/', 'https://perfectvape.com/pop-clouds/', 'https://perfectvape.com/sadboy/', 'https://perfectvape.com/shijin-vapor/', 
         'https://perfectvape.com/sour-house/', 'https://perfectvape.com/tornado-alley/', 'https://perfectvape.com/tropic-king/', 'https://perfectvape.com/twist-e-liquids/', 
         'https://perfectvape.com/pod-kits/', 'https://perfectvape.com/pod-kits/', 'https://perfectvape.com/voopoo-2/', 'https://perfectvape.com/geekvape-4/', 'https://perfectvape.com/lostvape/', 
         'https://perfectvape.com/oukitel/', 'https://perfectvape.com/rincoe-1/', 'https://perfectvape.com/batteries/', 'https://perfectvape.com/batteries/', 'https://perfectvape.com/ijoy-kit/', 
         'https://perfectvape.com/kanger-3/', 'https://perfectvape.com/lg/', 'https://perfectvape.com/nitecore-vape-battery-charger/', 'https://perfectvape.com/samsung/', 'https://perfectvape.com/sony/', 'https://perfectvape.com/mods/', 
         'https://perfectvape.com/mods/', 'https://perfectvape.com/sigelei/', 'https://perfectvape.com/smoktech-8/', 'https://perfectvape.com/starterkits/', 'https://perfectvape.com/starterkits/', 'https://perfectvape.com/eleaf-starter-kits/', 
         'https://perfectvape.com/freemax-starter-vape-kits/', 'https://perfectvape.com/geekvape-starter-kits/', 'https://perfectvape.com/juul-starter-kits/', 'https://perfectvape.com/smoktech-starter-kits/', 'https://perfectvape.com/suorin-starter-kits/', 'https://perfectvape.com/voopoo-starter-kits/', 'https://perfectvape.com/wismec-starter-kits/',
         'https://perfectvape.com/joyetech-starter-kits/', 'https://perfectvape.com/tanks/', 'https://perfectvape.com/tanks/', 'https://perfectvape.com/aspire-tanks/', 'https://perfectvape.com/freemax-tanks/', 'https://perfectvape.com/geekvape-tanks/', 'https://perfectvape.com/horizon-tanks/',
         'https://perfectvape.com/ijoy-tanks/', 'https://perfectvape.com/innokin-tanks/', 'https://perfectvape.com/ismoka-eleaf-tanks/', 'https://perfectvape.com/joyetech-tanks/', 'https://perfectvape.com/kanger-tanks/', 'https://perfectvape.com/smoktech-tanks/', 'https://perfectvape.com/uwell-tanks/', 'https://perfectvape.com/vaporesso-tanks/', 
         'https://perfectvape.com/accessories/', 'https://perfectvape.com/accessories/', 'https://perfectvape.com/ohm-volt-readers/', 'https://perfectvape.com/drip-tips/', 'https://perfectvape.com/adapters/', 'https://perfectvape.com/replacement-glass-for-tanks/', 'https://perfectvape.com/chargers/', 'https://perfectvape.com/coils-wires/', 'https://perfectvape.com/coils-wires/', 'https://perfectvape.com/aspire-coils/', 'https://perfectvape.com/eleaf-coils/', 
         'https://perfectvape.com/geekvape-coils/', 'https://perfectvape.com/suorin-coils/', 'https://perfectvape.com/Voopoo-coils/', 'https://perfectvape.com/horizon-coils/', 'https://perfectvape.com/joyetech-coils/', 'https://perfectvape.com/smoktech-coils/', 'https://perfectvape.com/uwell-coils/', 'https://perfectvape.com/kanger-coils/', 'https://perfectvape.com/nichrome-wire/', 'https://perfectvape.com/kanthal-wire/', 
         'https://perfectvape.com/titanium-wire/', 'https://perfectvape.com/beginner-vape-kits', 'https://perfectvape.com/alternative-devices/', 'https://perfectvape.com/alternative-devices/', 'https://perfectvape.com/cartridge-battery-devices/', 'https://perfectvape.com/cartridge-battery-devices/', 'https://perfectvape.com/airistech-2/', 'https://perfectvape.com/yocan/', 'https://perfectvape.com/leaf-buddi/', 'https://perfectvape.com/lookah-1/', 'https://perfectvape.com/loosie/', 'https://perfectvape.com/vapmod/', 'https://perfectvape.com/cartridges/', 'https://perfectvape.com/containers/', 'https://perfectvape.com/dry-kits/', 'https://perfectvape.com/oil-rigs/', 'https://perfectvape.com/nectar-collector/', 'https://perfectvape.com/rolling-trays/', 'https://perfectvape.com/silicone-collection/', 'https://perfectvape.com/water-pipes/', 'https://perfectvape.com/grinders/', 'https://perfectvape.com/wax-kits/', 'https://perfectvape.com/bulk-e-liquid-ready-to-vape/', 'https://perfectvape.com/flavor-concentrate-wholesale/', 'https://perfectvape.com/flavor-concentrate-wholesale/', 'https://perfectvape.com/capella-flavors/', 'https://perfectvape.com/flavor-concentrate-sale/', 'https://perfectvape.com/flavorwest-flavors-fw/', 'https://perfectvape.com/flavourart-flavors-fa/', 'https://perfectvape.com/lorann/', 'https://perfectvape.com/perfectvape-flavors/', 'https://perfectvape.com/theflavorapprentice-flavors-tfa/', 'https://perfectvape.com/nixodine-s/', 'https://perfectvape.com/unflavored-nicotine/', 'https://perfectvape.com/diy/', 'https://perfectvape.com/diy/', 'https://perfectvape.com/case-of-bottles/', 'https://perfectvape.com/pg-vg-drums/', 'https://perfectvape.com/bottles/', 'https://perfectvape.com/popular-vapes/', 'https://perfectvape.com/popular-vapes/', 'https://perfectvape.com/250ml-e-Liquid', 'https://perfectvape.com/blueberry-ice/', 'https://perfectvape.com/box-puff/', 'https://perfectvape.com/cartridge-battery-charger/', 'https://perfectvape.com/chocomint-vape-juice/', 'https://perfectvape.com/clear-vape-juice/', 'https://perfectvape.com/cookies-disposables/', 'https://perfectvape.com/custard-monster/', 'https://perfectvape.com/disposable-cart/', 'https://perfectvape.com/frozen-fruit-monster/', 'https://perfectvape.com/gummy-bear-vape/', 'https://perfectvape.com/ice-monster/', 'https://perfectvape.com/lava-flow/', 'https://perfectvape.com/lemonade-monster/', 'https://perfectvape.com/smok-disposable-vapes/', 'https://perfectvape.com/suorin-drop-vape/', 'https://perfectvape.com/510-thread-battery/', 'https://perfectvape.com/510-vape-pen', 'https://perfectvape.com/7000-puff-vape', 'https://perfectvape.com/air-bar-diamond/', 'https://perfectvape.com/apple-e-liquid-flavor', 'https://perfectvape.com/atomizer-vape', 'https://perfectvape.com/21700-battery/', 'https://perfectvape.com/best-nicotine-vape', 'https://perfectvape.com/best-tasting-vape-juice', 'https://perfectvape.com/blueberry-e-liquid', 'https://perfectvape.com/box-disposable-vapes', 'https://perfectvape.com/box-vape', 'https://perfectvape.com/caliburn-g-pods', 'https://perfectvape.com/caliburn-pods/', 'https://perfectvape.com/caliburn-vape/', 'https://perfectvape.com/candy-king-disposable', 'https://perfectvape.com/coconut-e-liquid', 'https://perfectvape.com/cola-e-liquid', 'https://perfectvape.com/danish-e-liquid', 'https://perfectvape.com/direct-vapor/', 'https://perfectvape.com/disposable-vapes-without-diacetyl', 'https://perfectvape.com/disposable-wax-vapes', 'https://perfectvape.com/disposable-cartridge-vapes', 'https://perfectvape.com/e-liquid-cartridge-boxes', 'https://perfectvape.com/eb-flavors/', 'https://perfectvape.com/eb-zero-nicotine', 'https://perfectvape.com/electronic-cigarette', 'https://perfectvape.com/empty-disposable-vapes', 'https://perfectvape.com/flum-pebble/', 'https://perfectvape.com/freemax-fireluke/', 'https://perfectvape.com/freemax-mesh-pro-coils/', 'https://perfectvape.com/fruity-pebbles-vape-juice', 'https://perfectvape.com/fryd-e-Liquid', 'https://perfectvape.com/funky-republic-vape/', 'https://perfectvape.com/geekvape-z-series-coil', 'https://perfectvape.com/grape-e-liquid', 'https://perfectvape.com/hyde-duo/', 'https://perfectvape.com/hyde-edge-rechargeable/', 'https://perfectvape.com/hyde-rave/', 'https://perfectvape.com/hyppe-bar', 'https://perfectvape.com/kros-nano', 'https://perfectvape.com/menthol/', 'https://perfectvape.com/mi-pod-vape', 'https://perfectvape.com/nicotine-free-puff-bar', 'https://perfectvape.com/hyppe-max-flow', 'https://perfectvape.com/nicotine-vape-cartridge', 'https://perfectvape.com/non-nicotine-vape-pen/', 'https://perfectvape.com/nord-5-coils', 'https://perfectvape.com/nord-pro', 'https://perfectvape.com/nord-x', 'https://perfectvape.com/pastel-cartel-vape', 'https://perfectvape.com/pineapple-e-liquid-flavor', 'https://perfectvape.com/pod-mesh', 'https://perfectvape.com/puff-bars/', 'https://perfectvape.com/rainbow-candy-eb', 'https://perfectvape.com/rare-disposable-vapes', 'https://perfectvape.com/raspberry-e-liquid', 'https://perfectvape.com/rda-vape', 'https://perfectvape.com/refillable-pod-system/', 'https://perfectvape.com/refillable-vape/', 'https://perfectvape.com/smok-nord/', 'https://perfectvape.com/smok-nord-5', 'https://perfectvape.com/smok-novo/', 'https://perfectvape.com/smok-rpm-5', 'https://perfectvape.com/smok-stick-v8', 'https://perfectvape.com/strawberry-e-liquid', 'https://perfectvape.com/suorin-air-pro', 'https://perfectvape.com/swift-flavors', 'https://perfectvape.com/unicorn-e-liquid', 'https://perfectvape.com/uwell-caliburn/', 'https://perfectvape.com/valyrian-3-tank', 'https://perfectvape.com/vanilla-e-liquid', 'https://perfectvape.com/vape-supplies', 'https://perfectvape.com/vapor-cigarette-products', 'https://perfectvape.com/vaporesso-xros-2', 'https://perfectvape.com/voopoo-drag/', 'https://perfectvape.com/voopoo-drag-4', 'https://perfectvape.com/wholesale-disposable-vapes', 'https://perfectvape.com/xros-3/', 'https://perfectvape.com/yocan-uni/', 'https://perfectvape.com/z-coils']
print(len(links), 'LINKS')



In [ ]:
import json
import os

# Specify the file path
json_file_path = 'perfectvape.json'

# Check if the file exists
if os.path.exists(json_file_path):
    # Open and read the JSON file
    with open(json_file_path, 'r') as file:
        try:
            found = json.load(file)
            print("JSON data:", len(found.keys()), 'items')
        except json.JSONDecodeError:
            print("Error: The file contains invalid JSON.")
else:
    found = dict()
    print(f"File '{json_file_path}' does not exist.")

In [5]:
from collections import Counter
import csv
product_list = list()
all_headers = list()
header_counter = Counter(all_headers)
product_tags = set()


def save():
    print('SAVING...')
    with open('perfectvape_scrape.csv', mode='w') as file:
        # Create a DictWriter object
        writer = csv.DictWriter(file, fieldnames=product_list[0].keys())

        # Write the header (column names)
        writer.writeheader()

        # Write the rows (each dictionary in the data list)
        writer.writerows(product_list)
        
    with open(json_file_path, 'w') as file:
        json.dump(found, file, indent=4) 


max_pages = 25
found_max = False
for link_index,l in enumerate(links):
    page = 1
    while True:
        if page > max_pages:
            break
        main_url = f'{l}?page={page}'
            
        try:
            if main_url in found:
                req_text = found[main_url]
            else:
                req_text = get_html(main_url)
                found[main_url] = req_text

            soup = BeautifulSoup(req_text)
            prods = soup.find_all('li', {'class': 'product'})
            

        except Exception as ex:
            print(ex, 'FAILED', main_url)
            break
           

        try:
            pagination = soup.find('ul', class_='pagination-list')

            # Get the last page number by finding the last <li> with an <a> tag that is not empty
            last_page = None
            for li in pagination.find_all('li'):
                a_tag = li.find('a')
                if a_tag and a_tag.text.strip().isdigit():
                    max_pages = int(a_tag.text.strip())
        except Exception as ex:
            max_pages = 25
            
        total_items = len(prods)
        print('LINK INDEX', link_index, 'OF', len(links), ' -- PAGE', page, 'OF', max_pages, f'({total_items})', main_url)

            
        page += 1
        

        
        if total_items == 0:
            break
            
        for psoup in prods:
            txt = psoup.get_text().strip()
            if txt == '':
                continue
                
            #print(psoup)

            # Extracting Product Name
            ptitle = psoup.find('p', {'class', 'card-title'}).find('a')
            # print(ptitle)
            product_name = ptitle.text.strip()

            # Extracting Brand Name
            brand_card = psoup.find('p', {'class', 'card-text--brand'})
            if brand_card:
                brand_name = brand_card.text.strip()
            else:
                brand_name = ''

            # Extracting Price
            price_card = psoup.find('span', {'class':'price--main'})
            if price_card:
                price = price_card.text.strip()
            else:
                price = ''

            # Extracting Nicotine Strength
            nicotine_strength = psoup.find('div', {'class', 'card-text--summary'}).text.strip()

            # Extracting Product URL
            full_link = ptitle['href']

            # Extracting Image URL
            image_url = psoup.find('img', {'class','card-image'})['src']

            # Print the extracted details
#             print("Product Name:", product_name)
#             print("Brand Name:", brand_name)
#             print("Price:", price)
#             print("Nicotine Strength:", nicotine_strength)
#             print("Product URL:", product_url)
#             print("Image URL:", image_url)
#             print()
#             print()

            tag = full_link.split('/')[-1]
            if tag == '':
                tag = full_link.split('/')[-2]
                
            print(tag, full_link)

            if tag in product_tags:
                continue
                
            try:
            
                if full_link in found:
                    reqtxt = found[full_link]
                else:
                    # print(full_link)
                    reqtxt = get_html(full_link, elements=False)
                    found[full_link] = reqtxt
            except Exception as ex:
                print(ex, 'FAILED', main_url)
                continue
                


                
            soup = BeautifulSoup(reqtxt)
            
            image_caro = soup.find('ul', {'class': 'productView-imageCarousel-nav'})
            if image_caro:
                image_urls = [img['src'].replace('100x100', '400x400') for img in image_caro.find_all('img')]
            else:
                image_urls = []



            description = ''
            warning = ''
            package_contents = ''
            ingredients = ''
            key_features = ''
            flavors = ''
            specifications = ''
            performance = ''
            disposable = ''
            why = ''
            preference = ''
            innovation = ''
            enjoyment = ''
            compatibility = ''
            coil = ''
            battery = ''
            puffs = ''
            nicotine = ''
            e_liquid = ''
            prefilled = ''
            devices = ''
            airflow = ''
            heating_element = ''
            operation = ''
            charging = ''
            screen = ''
            links_txt = ''
            colors = ''
            options_txt = ''
            returns = ''
            size = ''
            pods = ''
            hardware = ''
            wattage = ''
            faqs = ''

            section_strings_raw = {
                "FAQ's": "faqs",
                "FAQs": "faqs",
                "FAQ": "faqs",
                'Display Screen': 'screen',
                'Charging': 'charging',
                'Charging Port Info': 'charging',
                'Operation': 'operation',
                'Prefilled Capacity': 'prefilled',
                'Compatibility': 'compatibility',
                'Compatible with': 'compatibility',
                'Ingredients': 'ingredients',
                'INCLUDES': 'package_contents',
                'Kit Includes': 'package_contents',
                "What's Included": 'package_contents',
                'Package Content': 'package_contents',
                'Package Contents': 'package_contents',
                'Package Includes': 'package_contents',
                'Included in package': 'package_contents',
                'Included in the Package': 'package_contents',
                'Included': 'package_contents',
                'Includes': 'package_contents',
                'Contents': 'package_contents',
                "What’s Included": 'package_contents',
                'CALIFORNIA PROPOSITION 65 - Warning': 'warning',
                'CAUTION: this Product Contains Nicotine. Nicotine is an addictive chemical.': 'warning',
                'Warning': 'warning',
                'AVAILABLE OPTIONS': 'key_features',
                'Features and Specifications': 'key_features',
                'Key Features': 'key_features',
                'Features': 'key_features',
                'Flavors': 'flavors',
                'Flavor Selection': 'flavors',
                'Flavor Options': 'flavors',
                'Cutting-Edge Design and Technology:': 'key_features',
                'Superior Versatility and Performance:': 'key_features',
                'Flavor Selection:': 'flavors',
                'WARNING': 'warning',
                'Description': 'description',
                'Product Specifications': 'specifications',
                'Specifications': 'specifications',
                'Available Flavors': 'flavors',
                'Flavor Profiles': 'flavors',
                'Exceptional Flexibility and Performance': 'performance',
                'Disposable': 'disposable',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Tailored to Your Preference': 'preference',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Enjoy My Shisha on the Go': 'enjoyment',
                'The Ultimate Vaping Experience': 'enjoyment',
                'Elevate Your Vaping Experience': 'enjoyment',
                'Why Choose the Juice Head 30K Disposable?': 'why',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Features and Specifications': 'key_features',
                'Hotbox Features': 'key_features',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Experience the Ultimate Vaping Innovation': 'innovation',
                'Flavor Details': 'flavors',
                'Advanced Dual Mesh Coil': 'coil',
                'Coil Specifications': 'coil',
                'Battery Info': 'battery',
                'Puff Count': 'puffs',
                'Max Puffs': 'puffs',
                'What Is a Puff Count?': 'puffs',
                'Available Nicotine': 'nicotine',
                'Nicotine': 'nicotine',
                'Nicotine Strength': 'nicotine',
                'Battery Capacity': 'battery',
                'E-liquid Capacity': 'e_liquid',
                'Maximum Puffs': 'puffs',
                'Available Flavor Profiles': 'flavors',
                'Prefilled': 'prefilled',
                'Compatible Devices': 'devices',
                'Devices': 'devices',
                'California Proposition 65 Warning': 'warning',
                'Melli Flavors': 'flavors',
                'Airflow': 'airflow',
                'Heating Element': 'heating_element',
                'Quick Links': 'links',
                'Quick Link': 'links',
                'Colors': 'colors',
                'Color': 'colors',
                'Quick links': 'links',
                'Flavor': 'flavors',
                'Size': 'size',
                'Sizes': 'size',
                'Bottle Size': 'size',
                'Bottle Sizes': 'size',
                'Return and Exchange Policy Disclaimer:': 'returns',
                'Returns': 'returns',
                'Pods': 'pods',
                'Pod Info': 'pods',
                'Options': 'options',
                'Option': 'options',
                'Compatible Hardware': 'hardware',
                'Hardware': 'hardware',
                'Wattage Info': 'wattage',
                'Wattage': 'wattage',
                'Coil': 'coil',
                'Coils': 'coil',
                'Available Nicotine': 'nicotine',
                'How Are Disposables Different Than Others?': 'disposable',
            
            }
            
            functions = [
                str.upper,
                str.lower,
                str.capitalize,
                str.title,
                str.swapcase,
                str.casefold
            ]
            section_strings = dict()
            for k, v in section_strings_raw.items():
                key = k
                section_strings[key] = v
                if key.endswith(':'):
                    key = key.replace(':', '')
                    section_strings[key] = v
                else:
                    key = key.strip() + ':'
                    section_strings[key] = v
                for func in functions:
                    key = func(k)
                    section_strings[key] = v
                    if key.endswith(':'):
                        key = key.replace(':', '')
                        section_strings[key] = v
                    else:
                        key = key.strip() + ':'
                        section_strings[key] = v
                
            
            section_map = {
                'description': description,
                'warning': warning,
                'package_contents': package_contents,
                'ingredients': ingredients,
                'key_features': key_features,
                'flavors': flavors,
                'specifications': specifications,
                'performance': performance,
                'disposable': disposable,
                'why': why,
                'innovation': innovation,
                'preference': preference,
                'enjoyment': enjoyment,
                'compatibility': compatibility,
                'coil': coil,
                'battery': battery,
                'puffs': puffs,
                'nicotine': nicotine,
                'e_liquid': e_liquid,
                'prefilled': prefilled,
                'devices': devices,
                'airflow': airflow,
                'heating_element': heating_element,
                'operation': operation,
                'charging': charging,
                'screen': screen,
                'links': links_txt,
                'colors': colors,
                'returns': returns,
                'options': options_txt,
                'size': size,
                'pods': pods,
                'hardware': hardware,
                'wattage': wattage,
                'faqs': faqs
            }

            desc_section = soup.find('div', {'class': 'productView-descriptionWrapper'})
            if desc_section:
                cur_section = 'description'
                if desc_section:
                    headers = [tag.text for tag in desc_section.find_all(lambda tag: tag.name in ['strong','h1', 'h2', 'h3', 'h4', 'h5'])]
                    stext = desc_section.get_text(separator='\n').split('\n')
                    for s in stext:
                        s = s.strip()
                        if s == '':
                            continue
                        s_no_colon = s.replace(':', '').strip()
                        if s in section_strings:
                            cur_section = section_strings[s]
                        elif s_no_colon in section_strings:
                            cur_section = section_strings[s_no_colon]
                        elif s.lower().startswith('why') and s.endswith('?'):
                            cur_section = 'why'
                        elif 'features:' in s.lower():
                            cur_section = 'key_features'
                        elif s in headers:
                            all_headers.append(s)
                            header_counter = Counter(all_headers)
                            #if header_counter[s] > 5:
                            #    print(s, full_link)
                            cur_section = 'description'
                            section_map[cur_section] += f'{s}\n\n'
                        else:
                            section_map[cur_section] += f'{s}\n'


            #             print("Product Name:", product_name)
#             print("Brand Name:", brand_name)
#             print("Price:", price)
#             print("Nicotine Strength:", nicotine_strength)
#             print("Product URL:", product_url)
#             print("Image URL:", image_url)
            product_data = {
                'tag': tag,
                "title": product_name,
                'brand': brand_name,
                "link": full_link,
                "price": price,
                'summary': nicotine_strength,
                'image_url': image_url,
                'image_urls': image_urls

            }
            
            if image_url:
                download_image(image_url, f'{tag}', save_dir='perfectvape_images')
                
            if image_urls:
                image_num = 0
                for i in product_data['image_urls']:
                    if i and i != '':
                        download_image(i, f'{tag}-image{image_num}', save_dir='perfectvape_images')
                        image_num += 1
            
                    
            desc_fields = ''
            for s, v in section_map.items():
                if 'description' not in s:
                    s = s + '_description'
                product_data[s] = v.replace('\xa0', ' ').strip()
                if product_data[s] != '' and 'link' not in s:
                    desc_fields += f'\n{product_data[s]}'

            feats = list()
            feat = find_features(desc_fields)
            any_found, puffs_res, nico_res, ml_res, flav_text, dev_text = feat

            if any_found:
                #print(feat)
                feats.append(feat)
            disposable,recharge,battery,mesh,usb,adjustable,found_flavs = features_to_cats(feats)
            product_data['disposable'] = disposable
            product_data['rechargeable'] = recharge
            product_data['battery'] = battery
            product_data['mesh'] = mesh
            product_data['usb'] = usb
            product_data['adjustable'] = adjustable

            product_list.append(product_data)
            product_tags.add(tag)
        
            if len(product_list) % 10 == 0:
                rc = (random.choice(product_list))
                print('{')
                for key, value in rc.items():
                    if not value:
                        continue

                    if isinstance(value, list):
                        print(f"\t{key}: {len(value)} items")
                    else:
                        value = value.replace('\n', ' ')

                        if key == 'image_url' or key == 'link' or key == 'title'  or key == 'tag' :
                            print(f"\t{key}: {value}")
                        elif 1 <= len(value) <= 20:
                            print(f"\t{key}: {value}")
                        elif len(value) > 20:
                            print(f"\t{key}: {value[:20]}...")
                print('}')
        
                print(len(product_list), 'ITEMS FOUND')
                print()
                print()
                for item, count in header_counter.most_common(50):
                    if count > 5:
                        print(f"{item}: {count}")
                print()
                print()


        save()
    save()
save()

aspire-breeze-nxt-pod https://perfectvape.com/aspire-breeze-nxt-pod/
{
	tag: vaporesso-xros-nano-kit
	title: Vaporesso XROS NANO Kit
	brand: Vaporesso Vape Coils...
	link: https://perfectvape.com/vaporesso-xros-nano-kit/
	price: $24.66
	summary: With its refreshing ...
	image_url: https://cdn11.bigcommerce.com/s-nlylv/images/stencil/590x590/products/2286/6308/vaporesso-xros-nano-kit__62969.1713328047.jpg?c=2
	image_urls: 4 items
	description: With its refreshing ...
	battery: TRUE
	usb: TRUE
}
500 ITEMS FOUND


Parameters: 6


horizon-falcon-2-replacement-bulb-glass https://perfectvape.com/horizon-falcon-2-replacement-bulb-glass/
juice-head-cali-bars-300 https://perfectvape.com/juice-head-cali-bars-300/
uwell-caliburn-koko-pod-1-2ohm-4pk https://perfectvape.com/uwell-caliburn-koko-pod-1-2ohm-4pk/
horizon-falcon-2-replacement-coils-3pk https://perfectvape.com/horizon-falcon-2-replacement-coils-3pk/
smok-rpm80-pro-mod-pod-kit https://perfectvape.com/smok-rpm80-pro-mod-pod-kit/
sadboy-tfn

vapmod-vmod-magnet-adapter https://perfectvape.com/vapmod-vmod-magnet-adapter/
vapmod-magic-710-magnet-adapter https://perfectvape.com/vapmod-magic-710-magnet-adapter/
{
	tag: smok-tfv16-bulb-replacement-glass-tube
	title: Smok TFV16 Bulb Replacement Glass Tube
	brand: SMOKTech Vape Tanks,...
	link: https://perfectvape.com/smok-tfv16-bulb-replacement-glass-tube/
	price: $2.08
	summary: For the Smok TFV16 S...
	image_url: https://cdn11.bigcommerce.com/s-nlylv/images/stencil/590x590/products/1682/7491/smoktech-smok-tfv16-bulb-replacement-glass-tube__92842.1713327345.jpg?c=2
	description: For the Smok TFV16 S...
	links_description: Smok TFV16 Sub-Ohm T...
}
540 ITEMS FOUND


Parameters: 6


vaporesso-cascade-one-coils-3pk https://perfectvape.com/vaporesso-cascade-one-coils-3pk/
yocan-stix-battery-320mah-vv https://perfectvape.com/yocan-stix-battery-320mah-vv/
eleaf-mini-istick-kit-1050mah-10w-tc-box-mod https://perfectvape.com/eleaf-mini-istick-kit-1050mah-10w-tc-box-mod/
geekvape-cerberu

{
	tag: cali-bar-300
	title: Cali Bar Disposable 300 Puffs
	brand: Cali Grown Vapes
	link: https://perfectvape.com/cali-bar-300/
	price: $5.49
	summary: By California Grown ...
	image_url: https://cdn11.bigcommerce.com/s-nlylv/images/stencil/590x590/products/1863/7180/cali-grown-cali-bar-300__36247.1713327467.jpg?c=2
	description: By California Grown ...
}
590 ITEMS FOUND


Parameters: 7


voopoo-uforce-coils-5-pack https://perfectvape.com/voopoo-uforce-coils-5-pack/
suorin-ishare-starter-kit https://perfectvape.com/suorin-ishare-starter-kit/
eleaf-ijust-3-kit https://perfectvape.com/eleaf-ijust-3-kit/
joyetech-exceed-edge-coils-1-2-ohm-5pk https://perfectvape.com/joyetech-exceed-edge-coils-1-2-ohm-5pk/
joyetech-exceed-edge-aio-starter-kit https://perfectvape.com/joyetech-exceed-edge-aio-starter-kit/
smoktech-rolo-badge-pods-3pk https://perfectvape.com/smoktech-rolo-badge-pods-3pk/
joyetech-ego-aio-eco-coil-0-5ohm-5pcs-pack https://perfectvape.com/joyetech-ego-aio-eco-coil-0-5ohm-5pcs-

LINK INDEX 39 OF 268  -- PAGE 4 OF 9 (16) https://perfectvape.com/vapor-products/?page=4
horizon-binaries-cabin-10000 https://perfectvape.com/horizon-binaries-cabin-10000/
ijoy-bar-ic8000 https://perfectvape.com/ijoy-bar-ic8000/
so-soul-nola-bar-10000 https://perfectvape.com/so-soul-nola-bar-10000/
friobar-db7000 https://perfectvape.com/friobar-db7000/
smok-vape-pen-v2-glass-tube https://perfectvape.com/smok-vape-pen-v2-glass-tube/
geekvape-zeus-nano-2-glass-bulb https://perfectvape.com/geekvape-zeus-nano-2-glass-bulb/
snowwolf-ease-8000 https://perfectvape.com/snowwolf-ease-8000/
twist-6000 https://perfectvape.com/twist-6000/
privilege-100-club-tobacco-tfn-e-liquid-100ml https://perfectvape.com/privilege-100-club-tobacco-tfn-e-liquid-100ml/
bora-tfn-e-liquid-100ml https://perfectvape.com/bora-tfn-e-liquid-100ml/
cereal-series-tfn-e-liquid-100ml https://perfectvape.com/cereal-series-tfn-e-liquid-100ml/
frozen-series-tfn-e-liquid-100ml https://perfectvape.com/frozen-series-tfn-e-liquid-

LINK INDEX 39 OF 268  -- PAGE 11 OF 16 (16) https://perfectvape.com/vapor-products/?page=11
bc5000 https://perfectvape.com/bc5000/
imren-21700-5000mah-15a-2pk https://perfectvape.com/imren-21700-5000mah-15a-2pk/
geekvape-max100-aegis-max-2-kit https://perfectvape.com/geekvape-max100-aegis-max-2-kit/
hyde-i-d-4500 https://perfectvape.com/hyde-i-d-4500/
drule-prime-5-2500 https://perfectvape.com/drule-prime-5-2500/
uwell-caliburn-g2-mesh-coil-1-2ohm-4pk https://perfectvape.com/uwell-caliburn-g2-mesh-coil-1-2ohm-4pk/
uwell-caliburn-g2-pod-2pk https://perfectvape.com/uwell-caliburn-g2-pod-2pk/
hyde-n-bar-mini-2500 https://perfectvape.com/hyde-n-bar-mini-2500/
esco-bars-mega-5000 https://perfectvape.com/esco-bars-mega-5000/
the-milk-salt-nic-premium-e-liquid-30ml https://perfectvape.com/the-milk-salt-nic-premium-e-liquid-30ml/
the-milk-premium-e-liquid-100ml https://perfectvape.com/the-milk-premium-e-liquid-100ml/
lemonade-monster-salt-nic-premium-e-liquid-30ml https://perfectvape.com/lemon

...

LINK INDEX 197 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/box-disposable-vapes?page=2
SAVING...
LINK INDEX 198 OF 268  -- PAGE 1 OF 25 (6) https://perfectvape.com/box-vape?page=1
geekvape-t200-aegis-touch-mod https://perfectvape.com/geekvape-t200-aegis-touch-mod/
geekvape-l200-aegis-legend-2-mod https://perfectvape.com/geekvape-l200-aegis-legend-2-mod/
yocan-uni-pro-box-mod https://perfectvape.com/yocan-uni-pro-box-mod/
vapmod-magic-710-box-mod https://perfectvape.com/vapmod-magic-710-box-mod/
yocan-uni-box-mod https://perfectvape.com/yocan-uni-box-mod/
sigelei-50w-vr2-box-mod https://perfectvape.com/sigelei-50w-vr2-box-mod/
SAVING...
LINK INDEX 198 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/box-vape?page=2
SAVING...
LINK INDEX 199 OF 268  -- PAGE 1 OF 25 (4) https://perfectvape.com/caliburn-g-pods?page=1
uwell-caliburn-gz2-pod-kit https://perfectvape.com/uwell-caliburn-gz2-pod-kit/
uwell-caliburn-x-pods-2pk https://perfectvape.com/uwell-caliburn-x-pods-2pk/
uwell-cal

LINK INDEX 212 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/eb-zero-nicotine?page=2
SAVING...
LINK INDEX 213 OF 268  -- PAGE 1 OF 25 (8) https://perfectvape.com/electronic-cigarette?page=1
yocan-rex-kit https://perfectvape.com/yocan-rex-kit/
smok-vape-pen-v2-kit https://perfectvape.com/smok-vape-pen-v2-kit/
hyde-color-plus-1500 https://perfectvape.com/hyde-color-plus-1500/
hyde-original-plus-1300 https://perfectvape.com/hyde-original-plus-1300/
hyde-edge-edition-1500 https://perfectvape.com/hyde-edge-edition-1500/
loosie-max-battery-350mah https://perfectvape.com/loosie-max-battery-350mah/
yocan-evolve-plus-kit https://perfectvape.com/yocan-evolve-plus-kit/
kanger-protank-evod-mt32-replacement-coils-5pk https://perfectvape.com/kanger-protank-evod-mt32-replacement-coils-5pk/
SAVING...
LINK INDEX 213 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/electronic-cigarette?page=2
SAVING...
LINK INDEX 214 OF 268  -- PAGE 1 OF 25 (0) https://perfectvape.com/empty-disposable-vapes?pag

LINK INDEX 231 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/hyppe-max-flow?page=2
SAVING...
LINK INDEX 232 OF 268  -- PAGE 1 OF 25 (6) https://perfectvape.com/nicotine-vape-cartridge?page=1
uwell-caliburn-gz2-pod-kit https://perfectvape.com/uwell-caliburn-gz2-pod-kit/
smok-nord-4-pods-3pk https://perfectvape.com/smok-nord-4-pods-3pk/
voopoo-navi-cartridge-2pk https://perfectvape.com/voopoo-navi-cartridge-2pk/
uwell-caliburn-kit https://perfectvape.com/uwell-caliburn-kit/
joyetech-exceed-grip-cartridge-5pk https://perfectvape.com/joyetech-exceed-grip-cartridge-5pk/
suorin-air-cartridge-2ml-black https://perfectvape.com/suorin-air-cartridge-2ml-black/
SAVING...
LINK INDEX 232 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/nicotine-vape-cartridge?page=2
SAVING...
LINK INDEX 233 OF 268  -- PAGE 1 OF 25 (4) https://perfectvape.com/non-nicotine-vape-pen/?page=1
hyve-5000 https://perfectvape.com/hyve-5000/
twist-6000 https://perfectvape.com/twist-6000/
cloud-nurdz-4500 https://per

LINK INDEX 247 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/smok-nord/?page=2
SAVING...
LINK INDEX 248 OF 268  -- PAGE 1 OF 25 (6) https://perfectvape.com/smok-nord-5?page=1
smok-nord-50w-kit https://perfectvape.com/smok-nord-50w-kit/
smok-nord-50w-pods-3pk https://perfectvape.com/smok-nord-50w-pods-3pk/
smok-rpm2-mesh-coils-5pk https://perfectvape.com/smok-rpm2-mesh-coils-5pk/
smok-rpm-nord-pods-3pk https://perfectvape.com/smok-rpm-nord-pods-3pk/
smoktech-nord-replacement-pod-with-2-coils https://perfectvape.com/smoktech-nord-replacement-pod-with-2-coils/
smoktech-nord-coils https://perfectvape.com/smoktech-nord-coils/
SAVING...
LINK INDEX 248 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/smok-nord-5?page=2
SAVING...
LINK INDEX 249 OF 268  -- PAGE 1 OF 25 (13) https://perfectvape.com/smok-novo/?page=1
smok-novo-5-meshed-0-7ohm-mtl-pods-3pk https://perfectvape.com/smok-novo-5-meshed-0-7ohm-mtl-pods-3pk/
smok-novo-5-kit https://perfectvape.com/smok-novo-5-kit/
smok-novo-bar

LINK INDEX 259 OF 268  -- PAGE 2 OF 25 (0) https://perfectvape.com/vape-supplies?page=2
SAVING...
LINK INDEX 260 OF 268  -- PAGE 1 OF 25 (12) https://perfectvape.com/vapor-cigarette-products?page=1
geek-bar-pulse-15000 https://perfectvape.com/geek-bar-pulse-15000/
uwell-caliburn-gz2-pod-kit https://perfectvape.com/uwell-caliburn-gz2-pod-kit/
smok-novo-5-kit https://perfectvape.com/smok-novo-5-kit/
geek-bar-meloso-max-9000 https://perfectvape.com/geek-bar-meloso-max-9000/
uwell-caliburn-tenet-kit https://perfectvape.com/uwell-caliburn-tenet-kit/
vaporesso-xros-3-mini-kit https://perfectvape.com/vaporesso-xros-3-mini-kit/
vaporesso-xros-3-kit https://perfectvape.com/vaporesso-xros-3-kit/
vozol-star-6000 https://perfectvape.com/vozol-star-6000/
uwell-caliburn-x-kit https://perfectvape.com/uwell-caliburn-x-kit/
geekvape-max100-aegis-max-2-kit https://perfectvape.com/geekvape-max100-aegis-max-2-kit/
uwell-caliburn-ak2-kit https://perfectvape.com/uwell-caliburn-ak2-kit/
uwell-caliburn-kit ht